-----

Keyboard shortcuts:

| do this                  | by typing this   |
| ------------------------ |------------------|
| run cell                 | ⌃↩              |
| run cell and select next | ⇧↩             |

-----

In [1]:
from pprint import pprint

import urllib.request
import json
import csv
import codecs


class USZIPCodes:
    def __init__(self, data_url_prefix = 'https://raw.githubusercontent.com/yyu/GeoJSON-US/master'):
        self.data_url_prefix = data_url_prefix
        self.refresh_zipcode_latlons(f'{data_url_prefix}/ZIPCodesGazetteer.tsv')
        self.refresh_available_zipcodes(f'{data_url_prefix}/perZIPgeojson/all_zipcodes.txt')


    def refresh_zipcode_latlons(self, url):
        lines = [ line.decode('UTF8').strip() for line in urllib.request.urlopen(url).readlines() ]
        tsv = csv.DictReader(lines, delimiter='\t')
        self.gazetteer = dict((d['GEOID'], {'lat': float(d['INTPTLAT']), 'lon': float(d['INTPTLONG'])}) for d in tsv)


    def refresh_available_zipcodes(self, url):
        lines = [ zipcode.decode('UTF8').strip() for zipcode in urllib.request.urlopen(url).readlines() ]
        self.zipcode_list = lines[1:] # ignore the first line
        self.zipcode_set = set(self.zipcode_list)


    def make_point(self, zipcode):
        ll = self.gazetteer[zipcode]
        geofeature = {
            "type": "Feature",
            "properties": {
                "name": zipcode,
                "label": zipcode,
            },
            "geometry": {
                "type": "Point",
                "coordinates": [
                    ll['lon'],
                    ll['lat']
                ]
            }
        }
        return geofeature


    def make_url(self, zipcode):
        return f'{self.data_url_prefix}/perZIPgeojson/{zipcode[0]}/{zipcode[1]}/{zipcode[2]}/{zipcode}.json'


    def make_geojson_data(self, *zipcodes):
        d = {
            "type": "FeatureCollection",
            "crs": {
                "type": "name",
                "properties": {
                    "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
                }
            },
            "features": []
        }

        available_zipcodes = set(zipcodes) & self.zipcode_set
        print(available_zipcodes)

        for z in available_zipcodes:
            #point = self.make_point(z)
            url = self.make_url(z)

            try:
                s = urllib.request.urlopen(url).read()
            except urllib.error.URLError as e:
                print('failed to get ' + url, ':', e.reason)
                continue

            j = json.loads(s)

            d['features'].append(j)
            #d['features'].append(point)

        return d




In [2]:
us = USZIPCodes()

-----

In [3]:
#zipcodes = ('98109', '98121', '98040')

In [4]:
zipcodes = list(set('980%02d' % i for i in range(100)) & us.zipcode_set)

In [5]:
d = us.make_geojson_data(*zipcodes)

{'98077', '98092', '98059', '98051', '98047', '98058', '98045', '98029', '98033', '98027', '98036', '98074', '98087', '98043', '98070', '98040', '98002', '98001', '98004', '98032', '98052', '98003', '98034', '98014', '98019', '98005', '98028', '98065', '98056', '98031', '98012', '98055', '98068', '98021', '98026', '98042', '98024', '98006', '98007', '98037', '98020', '98038', '98072', '98053', '98008', '98075', '98039', '98030', '98010', '98011', '98022', '98023', '98050', '98057'}


In [6]:
for feature in d['features']:
    feature['properties']['style'] = {'color':'#0000ff', 'weight': .5, 'fillColor':'#000077', 'fillOpacity':0.2}

latlons = { z: [us.gazetteer[z]['lat'], us.gazetteer[z]['lon']] for z in zipcodes }

In [7]:
from string import Template
from ipywidgets import Label, HTML
import ipyleaflet

In [8]:
text_template = Template('''<div>ZIP Code
                                <ul class='list-group'>
                                    <li class='list-group-item'>$zipcode</li>
                                </ul>
                            </div>''')

texts = { z: text_template.substitute(zipcode=z) for z in zipcodes }

In [9]:
def handle_interaction(**kwargs):
    '''mouse interaction handling'''
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))

In [10]:
# balloon with text
def make_marker(zipcode):
    marker = ipyleaflet.Marker(location=latlons[zipcode])
    marker.popup = HTML(value=texts[zipcode], placeholder='', description='')
    return marker

In [11]:
center = [ 47.621795, -122.334958 ]
zoom = 8

heatmap = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.Strava.All)
g = ipyleaflet.GeoJSON(data=d, hover_style={'fillColor': '#00aaff'}, name='ZIP Codes')
layers_control = ipyleaflet.LayersControl()

markers = [ make_marker(z) for z in zipcodes ]
marker_cluster = ipyleaflet.MarkerCluster(markers=markers, name='Balloons')

# label showing [lat, lon]
label = Label()
display(label)

m = ipyleaflet.Map(center=center, zoom=zoom, basemap=ipyleaflet.basemaps.OpenMapSurfer.Roads)
m.on_interaction(handle_interaction)
#m.add_layer(heatmap)
m.add_layer(g)
m.add_control(layers_control)
m += marker_cluster

m

Label(value='')

Map(basemap={'url': 'http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}', 'max_zoom': 20, 'attribution': 'Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', 'name': 'OpenMapSurfer.Roads'}, center=[47.621795, -122.334958], controls=(LayersControl(),), layers=(TileLayer(attribution='Imagery from <a href="http://giscience.uni-hd.de/">GIScience Research Group @ University of Heidelberg</a> &mdash; Map data &copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>', base=True, max_zoom=20, min_zoom=1, name='OpenMapSurfer.Roads', options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size'], url='http://korona.geog.uni-heidelberg.de/tiles/roads/x={x}&y={y}&z={z}'), GeoJSON(data={'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'ZCTA5CE10': '98077', 'GEOID10': '98077', 'CLASSFP10': 'B5', 'MTFCC10': 'G6350', 'FUNCSTAT10': 'S', 'ALAND10': 44875573, 'AWATER10': 581839, 'INTPTLAT10': '+47.7523786', 'INTPTLON10': '-122.0602825', 'style': {'color': '#0000ff', 'weight': 0.5, 'fillColor': '#000077', 'fillOpacity': 0.2}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-122.102721, 47.769473999999995], [-122.102728, 47.769571], [-122.10272, 47.77006], [-122.10272, 47.770638], [-122.10270000000001, 47.770739999999996], [-122.100751, 47.770319], [-122.099643, 47.770576999999996], [-122.100022, 47.772456999999996], [-122.099515, 47.775667], [-122.099514, 47.775706], [-122.099514, 47.775872], [-122.10123899999999, 47.77588], [-122.101475, 47.775881], [-122.102226, 47.775884999999995], [-122.105045, 47.775897], [-122.105825, 47.7759], [-122.10717, 47.775424], [-122.109183, 47.775915999999995], [-122.109272, 47.776075999999996], [-122.109272, 47.776317999999996], [-122.109272, 47.777373], [-122.109211, 47.777708999999994], [-122.109211, 47.778835], [-122.109211, 47.779029], [-122.109348, 47.779661999999995], [-122.109439, 47.780105999999996], [-122.109746, 47.780687], [-122.10997499999999, 47.781144999999995], [-122.110355, 47.78152], [-122.110691, 47.781836999999996], [-122.111399, 47.782506999999995], [-122.111702, 47.782905], [-122.111864, 47.783179999999994], [-122.112079, 47.783715], [-122.112084, 47.783884], [-122.114043, 47.786223], [-122.114648, 47.78695], [-122.114792, 47.787155999999996], [-122.115175, 47.78762], [-122.115194, 47.787659999999995], [-122.11532700000001, 47.787800000000004], [-122.115475, 47.787938], [-122.11549600000001, 47.787956], [-122.115952, 47.78835], [-122.116106, 47.788492999999995], [-122.116562, 47.788919], [-122.116693, 47.789075999999994], [-122.116814, 47.789237], [-122.116911, 47.789403], [-122.116984, 47.789570999999995], [-122.117035, 47.789741], [-122.117047, 47.789837], [-122.119716, 47.789882999999996], [-122.12141, 47.789898], [-122.122361, 47.789884], [-122.122308, 47.791467999999995], [-122.122271, 47.791478], [-122.122258, 47.791556], [-122.122265, 47.791663], [-122.122265, 47.791748999999996], [-122.122266, 47.791832], [-122.122265, 47.791847], [-122.122246, 47.792355], [-122.12224, 47.792702999999996], [-122.122258, 47.79288], [-122.122251, 47.793073], [-122.12223900000001, 47.793482999999995], [-122.122237, 47.793530999999994], [-122.122232, 47.793678], [-122.122231, 47.793727], [-122.121979, 47.793768], [-122.121762, 47.793824], [-122.12155, 47.793893], [-122.121452, 47.793938], [-122.12124, 47.794004], [-122.12116499999999, 47.794031], [-122.121034, 47.794078999999996], [-122.120936, 47.794121999999994], [-122.120511, 47.794255], [-122.120548, 47.794109], [-122.120419, 47.794092], [-122.120161, 47.794062], [-122.119772, 47.794028], [-122.119511, 47.794013], [-122.119119, 47.794003], [-122.11885699999999, 47.794003], [-122.118334, 47.794022999999996], [-122.11819299999999, 47.794033], [-122.11

-----